In [2]:
import ast
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

### Funciones para ETL de credits.csv

In [3]:
credits_path = r'C:\Users\felip\Desktop\Stuff\Cursos\SoyHenry\Clases\LABS\PI ML Ops\pi_mlops_felipeviera\venv\Datasets\credits.csv'

In [4]:
def extraer_actores_directores(path_csv):
    # Se lee el archivo csv en chunks (pedazos) para manejar el gran volumen de datos
    chunk_size = 100000 # Ajustar según necesidades
    actors_data = []
    directors_data = []

    for chunk in pd.read_csv(path_csv, chunksize=chunk_size): # Itera sobre cada chunk del archivo csv
        for _, row in chunk.iterrows(): # Convierte cada chunk en un iterable para iterar por cada fila
            movie_id = row['id'] # Extrae el id de la película

            try: 
                # De forma segura extrae las string correspondientes al cast y al crew. 
                # Si alguno de estos campos es NaN devuelve una lista vacía
                # ast.literal_eval() evalua strings que contengan python literals (Ej: una lista de diccionarios)
                cast_list = ast.literal_eval(row['cast']) if pd.notna(row['cast']) else []
                crew_list = ast.literal_eval(row['crew']) if pd.notna(row['crew']) else []
            
            # Si ast.literal_eval() se encuentra con sintaxis no válida, 
            # el bloque except permitirá que el proceso no crashee y se pase a la siguiente fila
            except (ValueError, SyntaxError):
                continue 

            # Extrae actores desds cast_list
            for person in cast_list:
                if 'cast_id' in person: # Chequea si la persona es un actor
                    actors_data.append({'movie_id': movie_id, 
                                         'actor_name' : person.get('name', '') # Si "name" no se encuentra devolverá un string vacío
                                         })
                    
            # Extrae directores desde crew_list
            for person in crew_list:
                if person.get('job', '').lower() == 'director': # Se asegura que la string "Director" mantenga un formato uniforme
                    directors_data.append({
                        'movie_id' : movie_id,
                        'director_name' : person.get('name', '')
                    })

    actors_df = pd.DataFrame(actors_data)
    directors_df = pd.DataFrame(directors_data)

    return actors_df, directors_df

In [5]:
actors_df, directors_df = extraer_actores_directores(credits_path)

### EDA

In [14]:
actors_df.head()

,movie_id,actor_name
0,862,Tom Hanks
1,862,Tim Allen
2,862,Don Rickles
3,862,Jim Varney
4,862,Wallace Shawn


In [13]:
directors_df.head()

,movie_id,director_name
0,862,John Lasseter
1,8844,Joe Johnston
2,15602,Howard Deutch
3,31357,Forest Whitaker
4,11862,Charles Shyer


In [8]:
actors_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 562474 entries, 0 to 562473
Data columns (total 2 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   movie_id    562474 non-null  int64 
 1   actor_name  562474 non-null  object
dtypes: int64(1), object(1)
memory usage: 8.6+ MB


In [9]:
directors_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49048 entries, 0 to 49047
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   movie_id       49048 non-null  int64 
 1   director_name  49048 non-null  object
dtypes: int64(1), object(1)
memory usage: 766.5+ KB


In [10]:
print(actors_df.isna().sum())
print(directors_df.isna().sum())

movie_id      0
actor_name    0
dtype: int64
movie_id         0
director_name    0
dtype: int64


In [11]:
# Actores que además han sido directores

act_y_dir_df = pd.merge(
    actors_df,
    directors_df,
    left_on='actor_name',
    right_on='director_name',
    how='inner'
)
act_y_dir_df.tail(10)

,movie_id_x,actor_name,movie_id_y,director_name
135476,188421,Jimmy Sangster,31472,Jimmy Sangster
135477,75608,Robert Gardner,84001,Robert Gardner
135478,75608,Robert Gardner,75608,Robert Gardner
135479,75608,Robert Gardner,275331,Robert Gardner
135480,75608,Robert Gardner,257472,Robert Gardner
135481,75608,Robert Gardner,323132,Robert Gardner
135482,75608,Robert Gardner,276895,Robert Gardner
135483,258907,Barbara Hammer,258907,Barbara Hammer
135484,222848,Ben Ramsey,57351,Ben Ramsey
135485,222848,Ben Ramsey,22164,Ben Ramsey


In [12]:
# Exportar DataFrames a archivos csv

actors_df.to_csv(r'C:\Users\felip\Desktop\Stuff\Cursos\SoyHenry\Clases\LABS\PI ML Ops\pi_mlops_felipeviera\datasets\movies_actors.csv', index=False)

directors_df.to_csv(r'C:\Users\felip\Desktop\Stuff\Cursos\SoyHenry\Clases\LABS\PI ML Ops\pi_mlops_felipeviera\datasets\movies_directors.csv', index=False)